# BioMedBERT BigQuery Data Analysis/ Pre-training

In [ ]:
# imports
import os
import json
import numpy as np
import pandas as pd
import textwrap
import tensorflow as tf
from google.cloud import bigquery

In [ ]:
project_id = 'ai-vs-covid19'
client = bigquery.Client(project=project_id)

## Query Analysis

In [ ]:
# Get number of rows
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `ai-vs-covid19.BigBioMedBERT2.ncbi_comm_use`''').to_dataframe().total
row_count

In [ ]:
# get column names
col_names = client.query('''
  SELECT column_name
  FROM `ai-vs-covid19.BigBioMedBERT2`.INFORMATION_SCHEMA.COLUMNS
  WHERE table_name = 'ncbi_comm_use'
''').to_dataframe()
col_names

In [ ]:
# get general table information schema
table_schema = client.query('''
  SELECT *
  FROM `ai-vs-covid19.BigBioMedBERT2`.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
  WHERE table_name = 'ncbi_comm_use'
''').to_dataframe()
table_schema

In [ ]:
# select first 10 rows
first_10_rows = client.query('''
  SELECT *
  FROM `ai-vs-covid19.BigBioMedBERT2.ncbi_comm_use`
  LIMIT 10
''').to_dataframe()
first_10_rows

In [ ]:
# print body text
def print_body(body_series: pd.Series) -> str:
  print(textwrap.fill(body_series))

In [ ]:
print_body(first_10_rows.Body[3])

## Data Preprocessing

In [ ]:
!pip install sentencepiece
# !git clone https://github.com/google-research/bert

In [ ]:
# base imports
import os
import sys
import nltk
import sentencepiece as spm

In [ ]:
#import bert modules
sys.path.append("bert")
from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

### Preprocess text
Remove punсtuation, uppercase letters and non-utf symbols.

In [ ]:
regex_tokenizer = nltk.RegexpTokenizer("\w+")

def normalize_text(text):
  # lowercase text
  text = str(text).lower()
  # remove non-UTF
  text = text.encode("utf-8", "ignore").decode()
  # remove punctuation symbols
  text = " ".join(regex_tokenizer.tokenize(text))
  return text

In [ ]:
# example to normalize text from ``Body`` in BioMedBERT dataset
print_body(normalize_text(first_10_rows.Body[1]))

## Create Expanded csv dataset

In [ ]:
import os
import glob
import pandas as pd
from google.cloud import storage

In [ ]:
storage_client = storage.Client(project=project_id)

In [ ]:
bucket=storage_client.get_bucket('big_bio_med_bert_dump_csv')
# List all objects that satisfy the filter.
blobs=bucket.list_blobs(prefix='ncbi_comm_use')

In [ ]:
blob = [blob for blob in blobs]

In [ ]:
print(len(blob))
print(len(blob) // 5)
print((len(blob) // 5)*5)

In [ ]:
# Create a function called "chunks" with two arguments, l and n:
def split_list(data, chunk):
    # For item i in a range that is a length of data (l),
    for i in range(0, len(data), chunk):
        # Create an index range for data of chunk (e.g. 5) items:
        yield data[i:i+chunk]

In [ ]:
# list of length in which we have to split 
blob_split_1, blob_split_2, blob_split_3, blob_split_4, blob_split_5 = list(split_list(blob, 50))

In [ ]:
print(len(blob))
print(len(blob_split_1))
print(len(blob_split_2))
print(len(blob_split_3))
print(len(blob_split_4))
print(len(blob_split_5))

In [ ]:
blob_split_1[0:5]

In [ ]:
blob_split_2[0:5]

In [ ]:
def download_to_local(folder, blob_lst):
    print('File download Started…. Wait for the job to complete.')
    # Create this folder locally if not exists
    if not os.path.exists(folder):
        os.makedirs(folder)
    # Iterating through for loop one by one using API call
    for blob in blob_lst:
#         print('Blobs: {}'.format(blob.name))
        destination_uri = '{}/{}'.format(folder, (blob.name).split('/')[-1])
        blob.download_to_filename(destination_uri)
        print('Exported {} to {}'.format(blob.name, destination_uri))

In [ ]:
# !rm -rf data #data_1 data_2 data_3 data_4 #data_5
# !rm ncbi_comm_use_csv_A.csv

In [ ]:
# download first part of csv's
# download_to_local('data', blob)
# download_to_local('data_1', blob_split_1)
# download_to_local('data_2', blob_split_2)
# download_to_local('data_3', blob_split_3)
# download_to_local('data_4', blob_split_4)
# download_to_local('data_5', blob_split_5)

In [ ]:
# make combined csv
def combined_csv(data_folder):
    extension = 'csv'
    all_filenames = [i for i in glob.glob('{}/*.{}'.format(data_folder, extension))]
    #combine all files in the list
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
    return combined_csv

In [ ]:
# blob_csv_A = combined_csv('data')
# blob_csv = combined_csv('data')

In [ ]:
# len(blob_csv)

In [ ]:
# blob_csv.to_csv('gs://ekaba-assets/ncbi_comm_use.csv')

In [ ]:
# blob_csv_A.to_csv( "ncbi_comm_use_csv_A.csv", index=False, encoding='utf-8-sig')
# blob_csv.to_csv( "ncbi_comm_use.csv", index=False, encoding='utf-8-sig')

In [ ]:
# copy files from gcs bucket
# !gsutil -m cp gs://ekaba-assets/ncbi_comm_use_BODY.csv .

In [ ]:
# body = pd.read_csv('ncbi_comm_use.csv')

In [ ]:
# body_sel = body[['Body']]

In [ ]:
# body_sel.to_csv('gs://ekaba-assets/ncbi_comm_use_BODY.csv')

In [ ]:
# body_sel.to_csv( "ncbi_comm_use_BODY.csv", index=False, encoding='utf-8-sig')

In [ ]:
# remove FULL ncbi_comm_use.csv
# !rm ncbi_comm_use.csv

In [ ]:
# convert csv to txt
import csv
import sys
maxInt = sys.maxsize
csv.field_size_limit(maxInt)

csv_file = 'ncbi_comm_use_BODY.csv'
txt_file = 'ncbi_comm_use_BODY.txt'
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [ ]:
# move text file to GCS
# !gsutil -m cp ncbi_comm_use_BODY.txt gs://ekaba-assets/

In [ ]:
# remove csv file
# !rm ncbi_comm_use_BODY.csv

In [ ]:
from tensorflow.keras.utils import Progbar
def count_lines(filename):
  count = 0
  with open(filename) as fi:
    for line in fi:
      count += 1
  return count

In [ ]:
# Apply normalization to entire dataset
RAW_DATA_FPATH = "ncbi_comm_use_BODY.txt"
PRC_DATA_FPATH = "processed_ncbi_comm_use_BODY.txt"

# apply normalization to the dataset

total_lines = count_lines(RAW_DATA_FPATH)
bar = Progbar(total_lines)

with open(RAW_DATA_FPATH,encoding="utf-8") as fi:
  with open(PRC_DATA_FPATH, "w",encoding="utf-8") as fo:
    for l in fi:
      fo.write(normalize_text(l)+"\n")
      bar.add(1)

In [ ]:
# move processed text file to GCS
# !gsutil -m cp processed_ncbi_comm_use_BODY.txt gs://ekaba-assets/

In [ ]:
# remove intermediate files
# !rm ncbi_comm_use_BODY.csv ncbi_comm_use_BODY.txt #processed_ncbi_comm_use_BODY.txt

## Building the vocabulary

In [ ]:
PRC_DATA_FPATH = "processed_ncbi_comm_use_BODY.txt"
MODEL_PREFIX = "biomedbert" #@param {type: "string"}
VOC_SIZE = 32000 #@param {type:"integer"}
SUBSAMPLE_SIZE = 12800000 #@param {type:"integer"}
NUM_PLACEHOLDERS = 256 #@param {type:"integer"}

SPM_COMMAND = ('--input={} --model_prefix={} '
               '--vocab_size={} --input_sentence_size={} '
               '--shuffle_input_sentence=true ' 
               '--bos_id=-1 --eos_id=-1').format(
               PRC_DATA_FPATH, MODEL_PREFIX, 
               VOC_SIZE - NUM_PLACEHOLDERS, SUBSAMPLE_SIZE)

In [ ]:
spm.SentencePieceTrainer.Train(SPM_COMMAND)

In [ ]:
!ls

In [ ]:
!head -n 30 tokenizer.vocab

In [ ]:
def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

In [ ]:
import random

snt_vocab = read_sentencepiece_vocab("{}.vocab".format(MODEL_PREFIX))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

In [ ]:
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token

In [ ]:
bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

In [ ]:
ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
bert_vocab = ctrl_symbols + bert_vocab

In [ ]:
bert_vocab += ["[UNUSED_{}]".format(i) for i in range(VOC_SIZE - len(bert_vocab))]
print(len(bert_vocab))

In [ ]:
# write vocabulary to file
VOC_FNAME = "vocab.txt"

with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [ ]:
bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
bert_tokenizer.tokenize(first_10_rows.Body[0])[0:20]

## Generating pre-trained data

In [ ]:
# sharding the dataset
!mkdir ./shards
!split -a 4 -l 5560 -d $PRC_DATA_FPATH ./shards/shard_
!ls ./shards/

In [ ]:
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS = 20 #@param {type:"integer"}
DO_LOWER_CASE = True #@param {type:"boolean"}
PROCESSES = 2 #@param {type:"integer"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}

For each shard we need to call `create_pretraining_data.py` script

In [ ]:
XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
             "python3 bert/create_pretraining_data.py "
             "--input_file=./shards/{} "
             "--output_file={}/{}.tfrecord "
             "--vocab_file={} "
             "--do_lower_case={} "
             "--max_predictions_per_seq={} "
             "--max_seq_length={} "
             "--masked_lm_prob={} "
             "--random_seed=34 "
             "--dupe_factor=5")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                             VOC_FNAME, DO_LOWER_CASE, 
                             MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)

In [ ]:
tf.gfile.MkDir(PRETRAINING_DIR)
!$XARGS_CMD

In [ ]:
# !gsutil -m cp -r gs://ekaba-assets/pre_trained_data .

Save model assets and checkpoints to GCS

In [ ]:
BUCKET_NAME = "ekaba-assets"
MODEL_DIR = "bert_model"
tf.io.gfile.mkdir(MODEL_DIR)

Hyparameter configuration for BERT BASE

In [ ]:
VOC_SIZE = 32000
VOC_FNAME = "biomedbert-8M.txt"

In [ ]:
# use this for BERT-base

bert_base_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": VOC_SIZE
}

with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo:
    json.dump(bert_base_config, fo, indent=2)
  
# with open("{}/{}".format(MODEL_DIR, VOC_FNAME), "w") as fo:
#   for token in bert_vocab:
#     fo.write(token+"\n")

In [ ]:
!gsutil -m cp -r $MODEL_DIR gs://$BUCKET_NAME

In [ ]:
# if BUCKET_NAME:
#   !gsutil -m cp -r $MODEL_DIR $PRETRAINING_DIR gs://$BUCKET_NAME

# Train the BioMedBERT model

In [24]:
# !gcloud auth application-default login --no-launch-browser

In [ ]:
!pip install --user tensorflow==1.15.0

In [1]:
import os
import sys
import tensorflow as tf

In [2]:
tf.__version__

'1.15.0'

In [3]:
#import bert modules
sys.path.append("bert")
from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder

In [4]:
import logging
# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

In [5]:
BUCKET_NAME = "ekaba-assets" #@param {type:"string"}
MODEL_DIR = "bert_model" #@param {type:"string"}
PRETRAINING_DIR = "pre_trained_data" #@param {type:"string"}
VOC_FNAME = "biomedbert-8M.txt" #@param {type:"string"}

# Input data pipeline config
TRAIN_BATCH_SIZE = 128 #@param {type:"integer"}
MAX_PREDICTIONS = 20 #@param {type:"integer"}
MAX_SEQ_LENGTH = 128 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param

# Training procedure config
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
TRAIN_STEPS = 1000000 #@param {type:"integer"}
SAVE_CHECKPOINTS_STEPS = 2500 #@param {type:"integer"}
NUM_TPU_CORES = 8

if BUCKET_NAME:
    BUCKET_PATH = "gs://{}".format(BUCKET_NAME)
else:
    BUCKET_PATH = "."

BERT_GCS_DIR = "{}/{}".format(BUCKET_PATH, MODEL_DIR)
DATA_GCS_DIR = "{}/{}".format(BUCKET_PATH, PRETRAINING_DIR)

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME)
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json")

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR)

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.io.gfile.glob(os.path.join(DATA_GCS_DIR,'*tfrecord'))

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))
log.info("Using {} data shards".format(len(input_files)))

INFO:tensorflow:Using checkpoint: None
INFO:tensorflow:Using 10000 data shards


**Train on TPUs**

In [17]:
if 'TPU_NAME' in os.environ:
    log.info("Using TPU runtime")
    USE_TPU = True
    TPU_ADDRESS = 'grpc://' + '10.250.1.2:8470'

INFO:tensorflow:Using TPU runtime


In [25]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True)

tpu_cluster_resolver =  tf.distribute.cluster_resolver.TPUClusterResolver(
    tpu=TPU_ADDRESS, zone='us-central1-a', project='ai-vs-covid19', job_name='biomedbert')

run_config = tf.compat.v1.estimator.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.compat.v1.estimator.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.compat.v1.estimator.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.compat.v1.estimator.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

INFO:tensorflow:Using config: {'_task_id': 0, '_save_checkpoints_secs': None, '_experimental_max_worker_delay_secs': None, '_keep_checkpoint_max': 5, '_session_creation_timeout_secs': 7200, '_master': 'grpc://10.250.1.2:8470', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2e55aef978>, '_save_summary_steps': 100, '_global_id_in_cluster': 0, '_protocol': None, '_experimental_distribute': None, '_log_step_count_steps': None, '_eval_distribute': None, '_tpu_config': TPUConfig(iterations_per_loop=2500, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training_input_configuration=2, experimental_host_call_every_n_steps=1), '_cluster': <tensorflow.python.distribute.cluster_resolver.tpu_cluster_resolver.TPUClusterResolver object at 0x7f2e55aef160>, '_model_dir': 'gs://ekaba-assets/bert_model', '_evaluation_master': 'grpc://10.250.1.2:8470', '_num_ps_rep

In [26]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

INFO:tensorflow:Querying Tensorflow master (grpc://10.250.1.2:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:biomedbert/replica:0/task:0/device:CPU:0, CPU, -1, 9577806028284405766)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:biomedbert/replica:0/task:0/device:TPU:0, TPU, 17179869184, 2943474117103428884)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:biomedbert/replica:0/task:0/device:TPU:1, TPU, 17179869184, 5351861045237112242)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:biomedbert/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10694017606547298861)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:biomedbert/replica:0/task:0/device:TPU:3, TPU, 17179869184, 6358258477034075477)
INFO:tensorflow:*** Available Device: _DeviceAt

PermissionDeniedError: Error executing an HTTP request: HTTP response code 403 with body '{
  "error": {
    "code": 403,
    "message": "Insufficient Permission",
    "errors": [
      {
        "message": "Insufficient Permission",
        "domain": "global",
        "reason": "insufficientPermissions"
      }
    ]
  }
}
'
	 when initiating an upload to gs://ekaba-assets/bert_model/graph.pbtxt.tmp524db6a594ab44fbbd3c2ae492fb3c99